In [1]:
+import json
import torch
import torch.nn as nn
import random
from tqdm import tqdm
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [191]:
train_list = json.load(open('../data/qa_train_list.json'))
dev_list = json.load(open('../data/qa_dev_list.json'))
train_dict = json.load(open('../data/coqa-train-v1.0.json', encoding='utf8'))
dev_dict = json.load(open('../data/coqa-dev-v1.0.json', encoding='utf8'))

In [192]:
print(train_list[0])

In the text below two people are discussing a story.

Story:
The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. 

The Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. 

In March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. 

The Vatican Secret Archives

In [242]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
checkpoint = torch.load('save_small' + str(1))
model.load_state_dict(checkpoint['model_state_dict'])
model.cuda()
model.eval()

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [250]:
_question_prompt = '\nQ: '
_answer_prompt = '\nA: '


def get_text_up_to_question_number(text, number):
    pos = text.find(_answer_prompt)
    for _ in range(number):
        pos = text.find(_answer_prompt, pos + 1)
    return text[0:pos + 1]


def get_answers_number(text):
    return text.count(_answer_prompt)


def get_answer_number(text, number):
    pos = text.find(_answer_prompt)
    for _ in range(number):
        pos = text.find(_answer_prompt, pos + 1)
    end = text.find('\n', pos + len(_answer_prompt))
    return text[pos + len(_answer_prompt):end]


def get_question_number(text, number):
    pos = text.find(_question_prompt)
    for _ in range(number):
        pos = text.find(_question_prompt, pos + 1)
    end = text.find('\n', pos + len(_question_prompt))
    return text[pos + len(_question_prompt):end]


def get_all_answers(dev_dict, dev_index):
    answers = [[item['input_text'] for item in dev_dict['data'][dev_index]['answers']]]
    answers += [[item['input_text'] for item in dev_dict['data'][dev_index]['additional_answers'][str(index)]] for index in range(3)]
    return [list(set([answers[j][i] for j in range(len(answers))])) for i in range(len(answers[0]))]


def generate_answer(model, prompt):
    tokens = tokenizer.encode(prompt, return_tensors='pt')
    _length = 50
    tokens_length = tokens.shape[1]
    if tokens_length + _length > 1024:
        return ''
    output = model.generate(
             tokens.cuda(),
             max_length=tokens_length + _length,
             #temperature=0,
             pad_token_id=50256
    )
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    print(output)
    offset = len(prompt)
    start = offset + 1
    end = output.find('\n', start)
    return output[start:end].split(':')[-1].strip()


def get_text_from_data_item(item, max_num_questions=0, question_number=-1, last_question=True):
    text = 'In the text below two people are discussing a story.\n\n'
    text += 'Story:\n' + item['story'] + '\n\n'
    text += 'Discussion:\n'
    text += '\n'.join(['Q: ' + q['input_text'] 
                       + '\nA: ' + a['input_text'] 
                       for q, a in zip(item['questions'][max(0,question_number-max_num_questions):question_number+1], 
                                       item['answers'][max(0,question_number-max_num_questions):question_number+1]) 
                      ])
    if not last_question:
        text = '\n'.join(text.split('\n')[:-1]) + '\n'
        
    return text



In [195]:
print(dev_dict['data'][0]['questions'])

[{'input_text': 'What color was Cotton?', 'turn_id': 1}, {'input_text': 'Where did she live?', 'turn_id': 2}, {'input_text': 'Did she live alone?', 'turn_id': 3}, {'input_text': 'Who did she live with?', 'turn_id': 4}, {'input_text': 'What color were her sisters?', 'turn_id': 5}, {'input_text': 'Was Cotton happy that she looked different than the rest of her family?', 'turn_id': 6}, {'input_text': 'What did she do to try to make herself the same color as her sisters?', 'turn_id': 7}, {'input_text': 'Whose paint was it?', 'turn_id': 8}, {'input_text': "What did Cotton's mother and siblings do when they saw her painted orange?", 'turn_id': 9}, {'input_text': "Where did Cotton's mother put her to clean the paint off?", 'turn_id': 10}, {'input_text': 'What did the other cats do when Cotton emerged from the bucket of water?', 'turn_id': 11}, {'input_text': 'Did they want Cotton to change the color of her fur?', 'turn_id': 12}]


In [238]:
doc = 0
number = 0

text = get_text_from_data_item(dev_dict['data'][doc], max_num_questions=5, question_number=number-1, last_question=False)
#text += 'Q: What color was Cotton? Give me a reason why\n'
#text += 'Q: Where did she live? Give me a reason why\n'
text += 'Q: Where did she live?\n'
answer = generate_answer(model, text)
print(answer)

a barn near a farm house


In [239]:
## Interesting. Adding "give me a reason why improves accuracy?" test this!!!

In [248]:
doc = 0
number = 0

text = get_text_from_data_item(dev_dict['data'][doc], max_num_questions=5, question_number=number-1, last_question=False)
text += 'Q: What color was Cotton?\n'
text += 'A: white'
text += ' because '
#text += '\nQ: Why?\n'
answer = generate_answer(model, text)
#full_text = text + 'A: ' + answer
full_text = text + answer
tokens = tokenizer.encode(full_text, return_tensors='pt').cuda()
score = float(model(tokens, labels=tokens)[0])
print(full_text)
print(score)

In the text below two people are discussing a story.

Story:
Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. 

"What are you doing, Cotton?!" 

"I only wanted to be more like you". 

Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pret

In [199]:
doc = 0
number = 0

text = get_text_from_data_item(dev_dict['data'][doc], max_num_questions=5, question_number=number, last_question=False)
text += 'A: White'
#text += '\nReason: '
text += '\nQ: Are you sure?\n'
print(text)
answer = generate_answer(model, text)

In the text below two people are discussing a story.

Story:
Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. 

"What are you doing, Cotton?!" 

"I only wanted to be more like you". 

Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pret

In [200]:
answer

'No'

In [201]:
small_text = get_text_from_data_item(dev_dict['data'][doc], max_num_questions=5, question_number=number, last_question=False)
small_text = small_text[:-1] +  ' Give me a reason why\n'

In [202]:
print(small_text)

In the text below two people are discussing a story.

Story:
Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. 

"What are you doing, Cotton?!" 

"I only wanted to be more like you". 

Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pret

## Using the question "why?"

In [254]:
def compute_accuracy_of_model(model):
    total_number_of_questions = 0
    correct_answers = 0
    wrong_predictions = []

    false_positives = []
    dlist = dev_list[:10]
    for index, text in tqdm(enumerate(dlist), total=len(dlist)):
        total_questions = get_answers_number(text)
        all_answers = get_all_answers(dev_dict, index)
        for number in range(total_questions):
            small_text = get_text_from_data_item(dev_dict['data'][index], 
                                                 max_num_questions=5,
                                                 question_number=number,
                                                 last_question=False)
            prediction = generate_answer(model, small_text)
            small_text = small_text + 'A: ' + prediction + '\n'
            small_text = small_text[:-1] + '. The reason is'
            prediction = generate_answer(model, small_text)
            #print('PREDICTION', prediction)
            if not prediction:
                print('NO PREDICTION!!')
                continue
            prediction = prediction.replace('.', '').replace('"', '')
            it_was_answered = False
            for label in all_answers[number]:
                label = label.replace('.', '').replace('"', '')

                if prediction.lower() != 'unknown' and label.lower() == 'unknown':
                    false_positives.append(prediction)
                
                if prediction.lower() == label.lower():
                    correct_answers += 1
                    it_was_answered = True
                    break
                elif prediction.lower() in label.lower():
                    correct_answers += 1
                    it_was_answered = True
                    break
                elif label.lower() in prediction.lower():
                    correct_answers += 1
                    it_was_answered = True
                    break
                else:
                    wrong_predictions.append({'label': label, 'prediction': prediction})
            total_number_of_questions += 1

    return correct_answers/total_number_of_questions, wrong_predictions, false_positives

In [255]:
false_positives = None
wrong_answers = None

for index in range(10):
    checkpoint = torch.load('save_small' + str(index))
    model.load_state_dict(checkpoint['model_state_dict'])
    accuracy, wrong_answers, false_positives = compute_accuracy_of_model(model.cuda())
    print('Epoch:', index, ' with accuracy:', accuracy)

  0%|          | 0/10 [00:00<?, ?it/s]

In the text below two people are discussing a story.

Story:
Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. 

"What are you doing, Cotton?!" 

"I only wanted to be more like you". 

Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pret

In the text below two people are discussing a story.

Story:
Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. 

"What are you doing, Cotton?!" 

"I only wanted to be more like you". 

Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pret

In the text below two people are discussing a story.

Story:
Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. 

"What are you doing, Cotton?!" 

"I only wanted to be more like you". 

Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pret

In the text below two people are discussing a story.

Story:
Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. 

"What are you doing, Cotton?!" 

"I only wanted to be more like you". 

Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pret

...

In the text below two people are discussing a story.

Story:
(CNN) -- FBI agents on Friday night searched the Maryland home of the suspect in the recent disappearance of an American woman in Aruba, an agent said. 

The search is occurring in the Gaithersburg residence of Gary Giordano, who is currently being held in an Aruban jail, FBI Special Agent Rich Wolf told CNN. 

Agents, wearing vests that said FBI and carrying empty cardboard and plastic boxes, arrived about 8:40 p.m. Friday. About 15 unmarked cars could be seen on the street, as well as a Montgomery County police vehicle. 

Supervisory Special Agent Philip Celestini, who was at the residence, declined to comment further on the search, citing the active investigation. 

Aruban Solicitor General Taco Stein said earlier Friday that the suspect will appear in court Monday, where an investigating magistrate could order him held for at least eight more days, order him to remain on the island or release him outright due to a lack of

In the text below two people are discussing a story.

Story:
(CNN) -- FBI agents on Friday night searched the Maryland home of the suspect in the recent disappearance of an American woman in Aruba, an agent said. 

The search is occurring in the Gaithersburg residence of Gary Giordano, who is currently being held in an Aruban jail, FBI Special Agent Rich Wolf told CNN. 

Agents, wearing vests that said FBI and carrying empty cardboard and plastic boxes, arrived about 8:40 p.m. Friday. About 15 unmarked cars could be seen on the street, as well as a Montgomery County police vehicle. 

Supervisory Special Agent Philip Celestini, who was at the residence, declined to comment further on the search, citing the active investigation. 

Aruban Solicitor General Taco Stein said earlier Friday that the suspect will appear in court Monday, where an investigating magistrate could order him held for at least eight more days, order him to remain on the island or release him outright due to a lack of

In the text below two people are discussing a story.

Story:
(CNN) -- FBI agents on Friday night searched the Maryland home of the suspect in the recent disappearance of an American woman in Aruba, an agent said. 

The search is occurring in the Gaithersburg residence of Gary Giordano, who is currently being held in an Aruban jail, FBI Special Agent Rich Wolf told CNN. 

Agents, wearing vests that said FBI and carrying empty cardboard and plastic boxes, arrived about 8:40 p.m. Friday. About 15 unmarked cars could be seen on the street, as well as a Montgomery County police vehicle. 

Supervisory Special Agent Philip Celestini, who was at the residence, declined to comment further on the search, citing the active investigation. 

Aruban Solicitor General Taco Stein said earlier Friday that the suspect will appear in court Monday, where an investigating magistrate could order him held for at least eight more days, order him to remain on the island or release him outright due to a lack of

 70%|███████   | 7/10 [01:37<00:41, 13.94s/it]


KeyboardInterrupt: 

In [232]:
checkpoint = torch.load('save_small' + str(7))
model.load_state_dict(checkpoint['model_state_dict'])
accuracy, wrong_answers, false_positives = compute_accuracy_of_model(model.cuda())

100%|██████████| 10/10 [02:05<00:00, 12.59s/it]


In [234]:
wrong_answers

[{'label': 'in a barn', 'prediction': "all of the farmer's horses slept"},
 {'label': 'in a barn near', 'prediction': "all of the farmer's horses slept"},
 {'label': 'No',
  'prediction': 'her mommy and 5 other sisters were all orange'},
 {'label': 'no',
  'prediction': 'her mommy and 5 other sisters were all orange'},
 {'label': 'her mommy and 5 other sisters',
  'prediction': 'she was the only white one in the bunch'},
 {'label': 'with her mommy and 5 sisters',
  'prediction': 'she was the only white one in the bunch'},
 {'label': 'orange', 'prediction': 'her mommy and sisters found her'},
 {'label': 'orange with white tiger stripes',
  'prediction': 'her mommy and sisters found her'},
 {'label': 'orange and white',
  'prediction': 'her mommy and sisters found her'},
 {'label': 'No',
  'prediction': 'she wished she looked like the rest of her family'},
 {'label': 'no',
  'prediction': 'she wished she looked like the rest of her family'},
 {'label': 'paint herself like them',
  'predi

In [233]:
[print(item) for item in wrong_answers]

{'label': 'in a barn', 'prediction': "all of the farmer's horses slept"}
{'label': 'in a barn near', 'prediction': "all of the farmer's horses slept"}
{'label': 'No', 'prediction': 'her mommy and 5 other sisters were all orange'}
{'label': 'no', 'prediction': 'her mommy and 5 other sisters were all orange'}
{'label': 'her mommy and 5 other sisters', 'prediction': 'she was the only white one in the bunch'}
{'label': 'with her mommy and 5 sisters', 'prediction': 'she was the only white one in the bunch'}
{'label': 'orange', 'prediction': 'her mommy and sisters found her'}
{'label': 'orange with white tiger stripes', 'prediction': 'her mommy and sisters found her'}
{'label': 'orange and white', 'prediction': 'her mommy and sisters found her'}
{'label': 'No', 'prediction': 'she wished she looked like the rest of her family'}
{'label': 'no', 'prediction': 'she wished she looked like the rest of her family'}
{'label': 'paint herself like them', 'prediction': 'her mommy and sisters thought so

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

## Using "give me a reason"

In [221]:
def compute_accuracy_of_model(model):
    total_number_of_questions = 0
    correct_answers = 0
    wrong_predictions = []

    false_positives = []
    dlist = dev_list[:10]
    for index, text in tqdm(enumerate(dlist), total=len(dlist)):
        total_questions = get_answers_number(text)
        all_answers = get_all_answers(dev_dict, index)
        for number in range(total_questions):
            small_text = get_text_from_data_item(dev_dict['data'][index], 
                                                 max_num_questions=5,
                                                 question_number=number,
                                                 last_question=False)
            small_text = small_text[:-1] + ' Give me a reason why\n'
            prediction = generate_answer(model, small_text)
            #print('PREDICTION', prediction)
            if not prediction:
                print('NO PREDICTION!!')
                continue
            prediction = prediction.replace('.', '').replace('"', '')
            it_was_answered = False
            for label in all_answers[number]:
                label = label.replace('.', '').replace('"', '')

                if prediction.lower() != 'unknown' and label.lower() == 'unknown':
                    false_positives.append(prediction)
                
                if prediction.lower() == label.lower():
                    correct_answers += 1
                    it_was_answered = True
                    break
                elif prediction.lower() in label.lower():
                    correct_answers += 1
                    it_was_answered = True
                    break
                elif label.lower() in prediction.lower():
                    correct_answers += 1
                    it_was_answered = True
                    break
                else:
                    wrong_predictions.append({'label': label, 'prediction': prediction})
            total_number_of_questions += 1

    return correct_answers/total_number_of_questions, wrong_predictions, false_positives

In [210]:
false_positives = None
wrong_answers = None

for index in range(10):
    checkpoint = torch.load('save_small' + str(index))
    model.load_state_dict(checkpoint['model_state_dict'])
    accuracy, wrong_answers, false_positives = compute_accuracy_of_model(model.cuda())
    print('Epoch:', index, ' with accuracy:', accuracy)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0  with accuracy: 0.41843971631205673


100%|██████████| 10/10 [01:03<00:00,  6.34s/it]


Epoch: 1  with accuracy: 0.4397163120567376


100%|██████████| 10/10 [01:02<00:00,  6.26s/it]


...

Epoch: 7  with accuracy: 0.5390070921985816


100%|██████████| 10/10 [01:03<00:00,  6.30s/it]


Epoch: 8  with accuracy: 0.5602836879432624


100%|██████████| 10/10 [01:02<00:00,  6.26s/it]

Epoch: 9  with accuracy: 0.5460992907801419


In [222]:
checkpoint = torch.load('save_small' + str(7))
model.load_state_dict(checkpoint['model_state_dict'])
accuracy, wrong_answers, false_positives = compute_accuracy_of_model(model.cuda())

100%|██████████| 10/10 [01:03<00:00,  6.33s/it]


In [223]:
print(accuracy)

0.5390070921985816


In [225]:
[print(item) for item in wrong_answers]

{'label': 'No', 'prediction': 'yes'}
{'label': 'no', 'prediction': 'yes'}
{'label': 'No', 'prediction': 'because she wished she looked like the rest of her family'}
{'label': 'no', 'prediction': 'because she wished she looked like the rest of her family'}
{'label': 'paint herself like them', 'prediction': 'she wished she looked like the rest of her family'}
{'label': 'she painted herself', 'prediction': 'she wished she looked like the rest of her family'}
{'label': "the old farmer's", 'prediction': 'her mommy and sisters found her'}
{'label': 'the farmer', 'prediction': 'her mommy and sisters found her'}
{'label': "the farmer's", 'prediction': 'her mommy and sisters found her'}
{'label': 'started laughing', 'prediction': 'they laughed'}
{'label': 'they started laughing', 'prediction': 'they laughed'}
{'label': 'rubbed her face', 'prediction': 'they laughed'}
{'label': 'dropped her into a big bucket of water', 'prediction': 'she wanted to be more like her'}
{'label': 'into a big bucket 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

## Reference accuracy

In [208]:
false_positives = None
wrong_answers = None

for index in range(10):
    checkpoint = torch.load('save_small' + str(index))
    model.load_state_dict(checkpoint['model_state_dict'])
    accuracy, wrong_answers, false_positives = compute_accuracy_of_model(model.cuda())
    print('Epoch:', index, ' with accuracy:', accuracy)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0  with accuracy: 0.6808510638297872


100%|██████████| 10/10 [01:03<00:00,  6.35s/it]


Epoch: 1  with accuracy: 0.7092198581560284


100%|██████████| 10/10 [01:03<00:00,  6.31s/it]


...

Epoch: 7  with accuracy: 0.7163120567375887


100%|██████████| 10/10 [01:03<00:00,  6.32s/it]


Epoch: 8  with accuracy: 0.6950354609929078


100%|██████████| 10/10 [01:02<00:00,  6.25s/it]

Epoch: 9  with accuracy: 0.7092198581560284


In [ ]:
# get reason
# HSBC
# Kyle
# INLG2021